# RPKI statistics

https://lirportal.ripe.net/certification/content/static/statistics/ripencc.tal.txt

Also available from NRO adoption stats,
https://ftp.ripe.net/pub/stats/ripencc/nro-adoption/2023/07/01/rir-adoption.txt

In [14]:
import netaddr
import requests
import bz2
import time

from io import StringIO

import pandas as pd

In [4]:
def ipv4_prefix_from_row(row: pd.DataFrame) -> netaddr.IPRange:
    start = netaddr.IPAddress(row.prefix)
    return netaddr.IPRange(start, start + row.prefix_size)

def ipv6_prefix_from_row(row: pd.DataFrame) -> netaddr.IPNetwork:
    return f"{row.prefix}/{row.prefix_size}"


def rpki_cover_stats(date_str: str, roas_csv_url: str, delegated_stats_url: str):
    df_roas = pd.read_csv(roas_csv_url)

    prefixes = df_roas['IP Prefix']

    roa_space_v4 = netaddr.IPSet(prefixes[prefixes.str.contains("\.")])
    roa_space_v6 = netaddr.IPSet(prefixes[prefixes.str.contains(":")])

    assert prefixes[prefixes.str.contains(":")].size + prefixes[prefixes.str.contains("\.")].size == prefixes.size

    raw_delegated_extended = requests.get(delegated_stats_url).content
    raw_delegated_extended = StringIO(bz2.decompress(raw_delegated_extended).decode('ascii'))

    raw_delegated_extended.seek(0)
    df_delegated_extended = pd.read_csv(
        raw_delegated_extended,
        sep="|",
        skiprows=4,
        names=['rir', 'country', 'afi', 'prefix', 'prefix_size', 'date', 'status', 'uuid'],
        dtype={'rir': 'category', 'country': 'category',  'afi': 'category', 'prefix': str, 'size': int, 'date': str, 'status': 'category', 'uuid': str})

    df_delegated_extended.date = pd.to_datetime(df_delegated_extended.date, format='%Y%m%d')
    df_ripe_member_space = df_delegated_extended[(df_delegated_extended.status != 'available') & (df_delegated_extended.status != 'reserved')]


    df_ripe_member_space = df_delegated_extended[(df_delegated_extended.status != 'available') & (df_delegated_extended.status != 'reserved')]


    ipv4 = df_ripe_member_space[df_ripe_member_space.afi == 'ipv4'].apply(ipv4_prefix_from_row, axis=1, result_type='reduce')
    ipv6 = [netaddr.IPNetwork(p) for p in df_ripe_member_space[df_ripe_member_space.afi == 'ipv6'].apply(ipv6_prefix_from_row, axis=1)]

    ripe_space_v4 = netaddr.IPSet(ipv4)
    ripe_space_v6 = netaddr.IPSet(ipv6)
    
    print(date_str)
    print("ipv4 covered: {:.2%}".format((ripe_space_v4 & roa_space_v4).size/ripe_space_v4.size))
    print("ipv6 covered: {:.2%}".format((ripe_space_v6 & roa_space_v6).size/ripe_space_v6.size))
    
    return df_ripe_member_space

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:17: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:17: SyntaxWarning: invalid escape sequence '\.'
/var/folders/m4/qtvb8ff50ng0yw7xyt7t0n9c0000gp/T/ipykernel_38391/2961649739.py:14: SyntaxWarning: invalid escape sequence '\.'
  roa_space_v4 = netaddr.IPSet(prefixes[prefixes.str.contains("\.")])
/var/folders/m4/qtvb8ff50ng0yw7xyt7t0n9c0000gp/T/ipykernel_38391/2961649739.py:17: SyntaxWarning: invalid escape sequence '\.'
  assert prefixes[prefixes.str.contains(":")].size + prefixes[prefixes.str.contains("\.")].size == prefixes.size


In [5]:
rpki_cover_stats(
    "2023-07-01",
        "https://ftp.ripe.net/rpki/ripencc.tal/2023/07/01/roas.csv.xz",
    "https://ftp.ripe.net/pub/stats/ripencc/2023/delegated-ripencc-extended-20230701.bz2"
)

2023-07-01
ipv4 covered: 62.44%
ipv6 covered: 37.19%


,rir,country,afi,prefix,prefix_size,date,status,uuid
0,ripencc,PS,ipv4,1.178.112.0,4096,2007-11-26,allocated,08a0e97a-1d16-423d-9ed2-95c80e18d882
1,ripencc,PS,ipv4,1.178.128.0,4096,2007-11-26,allocated,08a0e97a-1d16-423d-9ed2-95c80e18d882
2,ripencc,PS,ipv4,1.178.208.0,4096,2010-06-25,allocated,08a0e97a-1d16-423d-9ed2-95c80e18d882
3,ripencc,ES,ipv4,1.178.224.0,8192,2010-06-25,allocated,7bdefdac-8071-46bf-9b36-e39850aca684
4,ripencc,PS,ipv4,1.179.40.0,2048,2009-05-18,allocated,08a0e97a-1d16-423d-9ed2-95c80e18d882
...,...,...,...,...,...,...,...,...
160161,ripencc,AM,ipv6,2001:7f9:8::,48,2016-06-30,assigned,37e73a9f-bdfc-42f4-bec0-71f89d498241
160162,ripencc,PL,ipv6,2001:7f9:c::,48,2017-01-31,assigned,e95fa156-2556-49c3-a9e6-f1cd669e776e
160163,ripencc,NL,ipv6,2001:7fb::,32,2007-06-05,assigned,db3b7f71-2600-4112-84bc-5ba6f642e97a
160164,ripencc,NL,ipv6,2001:7fd::,32,2003-08-29,assigned,db3b7f71-2600-4112-84bc-5ba6f642e97a


In [9]:
rpki_cover_stats(
    "2022-07-01",
    "https://ftp.ripe.net/rpki/ripencc.tal/2022/07/01/roas.csv.xz",
    "https://ftp.ripe.net/pub/stats/ripencc/2022/delegated-ripencc-extended-20220701.bz2"
)

2022-07-01
ipv4 covered: 54.75%
ipv6 covered: 34.17%


,rir,country,afi,prefix,prefix_size,date,status,uuid
0,ripencc,PS,ipv4,1.178.112.0,4096,2007-11-26,allocated,bbfbebd0-9563-4a11-855e-60cec4d81fa3
1,ripencc,PS,ipv4,1.178.128.0,4096,2007-11-26,allocated,bbfbebd0-9563-4a11-855e-60cec4d81fa3
2,ripencc,ES,ipv4,1.178.224.0,8192,2010-06-25,allocated,079128a3-990a-41ea-8c1a-a01eac9ecd43
3,ripencc,PS,ipv4,1.179.40.0,2048,2009-05-18,allocated,bbfbebd0-9563-4a11-855e-60cec4d81fa3
4,ripencc,PS,ipv4,1.179.72.0,2048,2009-05-18,allocated,bbfbebd0-9563-4a11-855e-60cec4d81fa3
...,...,...,...,...,...,...,...,...
157213,ripencc,PL,ipv6,2001:7f9:c::,48,2017-01-31,assigned,53b5a50f-3c28-454b-b2fc-43bc94bf364f
157214,ripencc,NL,ipv6,2001:7fb::,32,2007-06-05,assigned,c7ffc67b-1a1f-4e8d-a0df-21dedc60090e
157215,ripencc,JP,ipv6,2001:7fc:2::,47,2021-08-26,assigned,2b89ed19-f030-4c13-8a60-6dc4dd891215
157216,ripencc,NL,ipv6,2001:7fd::,32,2003-08-29,assigned,c7ffc67b-1a1f-4e8d-a0df-21dedc60090e


In [10]:
rpki_cover_stats(
    "2021-07-01",
    "https://ftp.ripe.net/rpki/ripencc.tal/2021/07/01/roas.csv.xz",
    "https://ftp.ripe.net/pub/stats/ripencc/2021/delegated-ripencc-extended-20210701.bz2"
)

2021-07-01
ipv4 covered: 48.77%
ipv6 covered: 32.49%


,rir,country,afi,prefix,prefix_size,date,status,uuid
0,ripencc,ES,ipv4,1.178.224.0,8192,2010-06-25,allocated,0e7d7e99-6beb-4efb-b563-dbec82b82aaf
1,ripencc,FR,ipv4,1.179.112.0,4096,2010-06-25,allocated,6c001cec-2f6f-47f7-bb6f-dc2fc81eee5d
2,ripencc,FR,ipv4,2.0.0.0,1048576,2010-07-12,allocated,315c3535-ea7d-40c4-a30a-83ab0ad0b9d9
3,ripencc,NL,ipv4,2.16.0.0,524288,2010-09-10,allocated,0e3542ac-459f-4f33-a181-b10fa116f197
4,ripencc,GB,ipv4,2.24.0.0,524288,2010-09-21,allocated,df92ab71-0c81-4a4a-b831-cf0077cd2a4d
...,...,...,...,...,...,...,...,...
151158,ripencc,PL,ipv6,2001:7f9:c::,48,2017-01-31,assigned,067a8e52-0b28-4160-a164-2fb37eac678f
151159,ripencc,NL,ipv6,2001:7fb::,32,2007-06-05,assigned,4f70b451-dc8b-4008-be72-e4feeff67e50
151160,ripencc,NL,ipv6,2001:7fc::,47,2021-01-04,assigned,8ad2acac-8d68-49ec-a500-19f5b2bb1f72
151161,ripencc,NL,ipv6,2001:7fd::,32,2003-08-29,assigned,4f70b451-dc8b-4008-be72-e4feeff67e50


In [11]:
rpki_cover_stats(
    "2020-07-01",
    "https://ftp.ripe.net/rpki/ripencc.tal/2020/07/01/roas.csv.xz",
    "https://ftp.ripe.net/pub/stats/ripencc/2020/delegated-ripencc-extended-20200701.bz2"
)

2020-07-01
ipv4 covered: 43.03%
ipv6 covered: 27.84%


,rir,country,afi,prefix,prefix_size,date,status,uuid
0,ripencc,FR,ipv4,2.0.0.0,1048576,2010-07-12,allocated,8723341c-c9ce-4caa-a152-383a3eced151
1,ripencc,EU,ipv4,2.16.0.0,524288,2010-09-10,allocated,27a50647-1087-48f1-bbfd-65e713610c3b
2,ripencc,GB,ipv4,2.24.0.0,524288,2010-09-21,allocated,09e4be75-86ae-4037-84d8-df638f9a5545
3,ripencc,IT,ipv4,2.32.0.0,1048576,2010-04-29,allocated,3e749ca3-b42b-47ba-ae0d-7e2917d86d80
4,ripencc,AE,ipv4,2.48.0.0,262144,2010-05-28,allocated,f6dfe57d-d793-48ee-a397-e6a7edc12ea7
...,...,...,...,...,...,...,...,...
146677,ripencc,AM,ipv6,2001:7f9:8::,48,2016-06-30,assigned,9078df3d-3e0f-4195-a27e-6a94df0750e2
146678,ripencc,PL,ipv6,2001:7f9:c::,48,2017-01-31,assigned,7721e04f-c047-4ab1-a4dd-fcfacd49d8fb
146679,ripencc,EU,ipv6,2001:7fb::,32,2007-06-05,assigned,1fa65c89-7cb3-4478-afd2-285105aa3480
146680,ripencc,EU,ipv6,2001:7fd::,32,2003-08-29,assigned,1fa65c89-7cb3-4478-afd2-285105aa3480


In [15]:
t0 = time.time()
df_ripe_member_space = rpki_cover_stats(
    "2023-07-01",
        "https://ftp.ripe.net/rpki/ripencc.tal/2023/07/01/roas.csv.xz",
    "https://ftp.ripe.net/pub/stats/ripencc/2023/delegated-ripencc-extended-20230701.bz2"
)
print(time.time() - t0)

2023-07-01
ipv4 covered: 62.44%
ipv6 covered: 37.19%
15.494277000427246


In [16]:
df_ripe_member_space

,rir,country,afi,prefix,prefix_size,date,status,uuid
0,ripencc,PS,ipv4,1.178.112.0,4096,2007-11-26,allocated,08a0e97a-1d16-423d-9ed2-95c80e18d882
1,ripencc,PS,ipv4,1.178.128.0,4096,2007-11-26,allocated,08a0e97a-1d16-423d-9ed2-95c80e18d882
2,ripencc,PS,ipv4,1.178.208.0,4096,2010-06-25,allocated,08a0e97a-1d16-423d-9ed2-95c80e18d882
3,ripencc,ES,ipv4,1.178.224.0,8192,2010-06-25,allocated,7bdefdac-8071-46bf-9b36-e39850aca684
4,ripencc,PS,ipv4,1.179.40.0,2048,2009-05-18,allocated,08a0e97a-1d16-423d-9ed2-95c80e18d882
...,...,...,...,...,...,...,...,...
160161,ripencc,AM,ipv6,2001:7f9:8::,48,2016-06-30,assigned,37e73a9f-bdfc-42f4-bec0-71f89d498241
160162,ripencc,PL,ipv6,2001:7f9:c::,48,2017-01-31,assigned,e95fa156-2556-49c3-a9e6-f1cd669e776e
160163,ripencc,NL,ipv6,2001:7fb::,32,2007-06-05,assigned,db3b7f71-2600-4112-84bc-5ba6f642e97a
160164,ripencc,NL,ipv6,2001:7fd::,32,2003-08-29,assigned,db3b7f71-2600-4112-84bc-5ba6f642e97a
